# [E-14]Movielens

## 데이터 준비와 전처리

In [2]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 분석해 봅시다

### ratings 에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

### ratings 에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

### 가장 인기있는 영화 30개 (인기순)

In [13]:
data = pd.merge(movies, ratings)
using_cols = ['user_id', 'title', 'counts']
data = data[using_cols]
data.head(10)

,user_id,title,counts
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [15]:
data['title'] = data['title'].str.lower()

movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다

In [17]:
favorite = ['terminator, the (1984)', 'toy story (1995)', 'matrix, the (1999)', 
            'pulp fiction (1994)', 'ghostbusters (1984)']

user = pd.DataFrame({'user_id' : ['kiim']*5, 'title' : favorite, 'counts' : [5]*5})

if not data.isin({'user_id' : ['kiim']})['user_id'].any():
    data = data.append(user)

data.tail(20)

,user_id,title,counts
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


### 전처리

In [19]:
# 고유한 유저, 제목을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

user_to_idx = {v:k for k, v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다.
print(user_to_idx['kiim'])
print(title_to_idx['pulp fiction (1994)'])

6039
283


In [20]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다.
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다.
else:
    print('user_id column indexing Fail!')
    
# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!')

data

user_id column indexing OK!
title column indexing OK!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1122,5
1,6039,0,5
2,6039,2325,5
3,6039,283,5


## CSR matrix 를 직접 만들어 봅시다

In [23]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [24]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [25]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, 
                                   iterations=30, dtype=np.float32)

In [26]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [27]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [28]:
kiim, pulp_fiction = user_to_idx['kiim'], title_to_idx['pulp fiction (1994)']
kiim_vector, pulp_fiction_vector = als_model.user_factors[kiim], als_model.item_factors[pulp_fiction]

In [29]:
kiim_vector

array([ 6.80094898e-01,  9.67151951e-03,  4.06058818e-01,  3.71326327e-01,
       -1.73615202e-01,  9.67005908e-01,  4.19849515e-01,  7.07215741e-02,
        5.45531220e-04,  3.80481392e-01, -1.24943733e-01, -1.47625327e-01,
       -4.79380846e-01,  4.08991754e-01,  1.26154459e+00, -5.33981919e-01,
       -4.35625792e-01, -3.95202935e-01,  2.58991212e-01,  5.44590533e-01,
       -4.80468899e-01,  1.19961195e-01, -5.95639169e-01, -3.87701213e-01,
       -1.39077500e-01,  3.60449791e-01, -6.98778391e-01, -3.87644559e-01,
        2.27056399e-01,  5.92027716e-02, -9.94823575e-02, -1.89295933e-01,
       -5.47365725e-01, -1.37062833e-01, -5.01883626e-01,  6.06560767e-01,
        1.03531033e-01, -1.20902091e-01,  3.79745573e-01, -2.26586476e-01,
       -5.46148717e-01, -8.17536488e-02,  7.47364402e-01, -5.85156918e-01,
       -5.92535675e-01, -1.61319524e-01, -5.89459717e-01,  1.09395206e+00,
       -2.33324975e-01,  6.12344146e-01, -1.09431811e-01,  6.02760203e-02,
       -5.49740553e-01, -

In [30]:
pulp_fiction_vector

array([ 9.69590154e-03,  9.52462293e-03,  5.43430150e-02,  2.53465958e-02,
       -5.80639066e-03,  1.51276244e-02,  2.64340714e-02,  2.34771948e-02,
       -1.15668774e-02,  1.42715650e-03,  7.84352701e-03, -5.59275923e-03,
        6.40712539e-03,  2.39305403e-02,  2.50843707e-02, -1.57291815e-02,
        1.47262705e-03,  6.87642116e-03,  1.15409335e-02,  2.56108586e-02,
        9.96147137e-05, -3.44542190e-02,  1.19308652e-02, -2.79108062e-02,
       -1.10010905e-02,  2.34090928e-02, -3.90790356e-03,  1.97645891e-02,
       -8.50399956e-04,  2.72513181e-02, -3.22962180e-04,  1.74372010e-02,
        3.25749032e-02, -1.26000047e-02,  1.68754149e-03,  2.31374800e-02,
       -2.01385152e-02,  2.79344618e-02,  1.06373774e-02,  2.31128260e-02,
        1.53368851e-02, -5.41146332e-03, -4.88965912e-03, -3.72545458e-02,
        7.63704826e-04, -2.98927873e-02, -1.92026049e-02,  3.87465842e-02,
        2.41849106e-02,  2.11392939e-02, -1.19580620e-03, -2.14932300e-02,
       -2.84709949e-02, -

## 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [31]:
np.dot(kiim_vector, pulp_fiction_vector)

0.40377474

In [33]:
other_movie = title_to_idx['e.t. the extra-terrestrial (1982)']
other_movie_vector = als_model.item_factors[other_movie]
np.dot(kiim_vector, other_movie_vector)

0.11162939

## 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [34]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [35]:
get_similar_title('pulp fiction (1994)')

['pulp fiction (1994)',
 'goodfellas (1990)',
 'fargo (1996)',
 'usual suspects, the (1995)',
 'shawshank redemption, the (1994)',
 'silence of the lambs, the (1991)',
 'reservoir dogs (1992)',
 'good will hunting (1997)',
 'l.a. confidential (1997)',
 'seven (se7en) (1995)']

In [36]:
get_similar_title('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 'babe (1995)',
 "bug's life, a (1998)",
 'aladdin (1992)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 "there's something about mary (1998)"]

## 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [38]:
user = user_to_idx['kiim']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

toy story 2 (1999) 0000.489
terminator 2: judgment day (1991) 0000.484
sixth sense, the (1999) 0000.481
usual suspects, the (1995) 0000.381
silence of the lambs, the (1991) 0000.345
back to the future (1985) 0000.327
fargo (1996) 0000.312
aliens (1986) 0000.304
reservoir dogs (1992) 0000.299
twelve monkeys (1995) 0000.298
alien (1979) 0000.292
groundhog day (1993) 0000.290
bug's life, a (1998) 0000.289
princess bride, the (1987) 0000.288
star wars: episode v - the empire strikes back (1980) 0000.287
fugitive, the (1993) 0000.284
shawshank redemption, the (1994) 0000.281
total recall (1990) 0000.280
raiders of the lost ark (1981) 0000.260
l.a. confidential (1997) 0000.257


In [44]:
rihanna = title_to_idx['terminator 2: judgment day (1991)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('matrix, the (1999)', 0.2760907432111592),
 ('terminator, the (1984)', 0.18572543309601172),
 ('toy story (1995)', 0.03342757240497053),
 ('pulp fiction (1994)', 0.00027773753415425323),
 ('ghostbusters (1984)', -0.015644472989724205)]

# 회고

이번 실습의 추천 시스템 모델 구성은 생각보다 간단했다.  
그런데 정확도는 높지 않은 거 같다.  
더 많은 영화 별점을 추가하면 정확도가 높아질까?  
더 좋은 추천 시스템 모델이 있는지 궁금해졌다.  